In [1]:
import pandas as pd
from copy import deepcopy

## Part 1, Attribution

In [2]:
# Read the data and find all cost types
allocation_data = pd.read_csv("attribution_allocation_student_data.csv")
allocation_data.fillna(0,inplace=True)
channel_spend = pd.read_csv("channel_spend_student_data.csv")
tiers_cost = list(channel_spend.iloc[:,1])
tiers_cost = [eval(tier_cost) for tier_cost in tiers_cost]
cost_types = list(tiers_cost[0].keys())

In [3]:
channel_spend

,Unnamed: 0,0
0,tier1,"{'email': 1000.0, 'social': 1000.0000000000001..."
1,tier2,"{'email': 2000.0, 'social': 2000.0, 'display':..."
2,tier3,"{'email': 3000.0, 'social': 3000.0, 'display':..."


In [4]:
# Create the subset of different convert condition and tiers
convert_TF = [True,False]
tiers = [1,2,3]
allocation_group = [[[0,dict()] for _ in tiers] for _ in convert_TF]
for i,convert in enumerate(convert_TF):
    for j,tier in enumerate(tiers):
        allocation_group[i][j] = allocation_data[(allocation_data["convert_TF"]==convert) & (allocation_data["tier"]==tier)].iloc[:,1:6]

In [5]:
# Compute the credit of each subset and find the total credit for each channel
def count(x,cost,option):
    start = 0
    end = len(x)-1
    while x[end]==0:
        end-=1
    if option==1:
        cost[x[end]]+=1
    elif option==2:
        if end-start<=1:
            cost[x[start]]+=0.5
            cost[x[end]]+=0.5
        else:
            cost[x[start]]+=0.4
            cost[x[end]]+=0.4
            size = end-start-1
            for i in range(size):
                cost[x[start+i+1]]+=0.2/size
    elif option==3:
        size = end-start+1
        for i in range(size):
            cost[x[start+i]] += 1/size

In [6]:
cost = dict()
for i in cost_types:
    cost[i] = 0
options = {"Last":1,"Position":2,"Linear":3}

number_cus_linear = [[deepcopy(cost) for _ in tiers] for _ in convert_TF]
number_cus_last = [[deepcopy(cost) for _ in tiers] for _ in convert_TF]
number_cus_position = [[deepcopy(cost) for _ in tiers] for _ in convert_TF]

In [7]:
# Compute the CAC for each tier each channel (linear)
option = options["Linear"]
for i,convert in enumerate(convert_TF):
    for j,tier in enumerate(tiers):
        allocation_group[i][j].apply(lambda x: count(x,number_cus_linear[i][j],option),axis=1)
# Compute the CAC for each tier each channel
CACs = [[0 for _ in cost_types] for _ in tiers]
for i in range(len(tiers)):
    for j,cost_type in enumerate(cost_types):
        CACs[i][j] = tiers_cost[i][cost_type]/number_cus_linear[0][i][cost_type]
CACs = pd.DataFrame(CACs,columns=cost_types)
CACs["tiers"] = [1,2,3]
CACs.set_index("tiers")

,email,social,display,paid_search,referral,organic_search,direct
tiers,,,,,,,
1,3.981949,2.063629,2.276521,7.809449,0.659152,0.0,0.0
2,4.519604,2.432005,2.762113,8.409840,0.752818,0.0,0.0
3,5.159515,3.014368,3.545610,8.849558,0.911840,0.0,0.0


In [8]:
# Compute the CAC for each tier each channel (last)
option = options["Last"]
for i,convert in enumerate(convert_TF):
    for j,tier in enumerate(tiers):
        allocation_group[i][j].apply(lambda x: count(x,number_cus_last[i][j],option),axis=1)
# Compute the CAC for each tier each channel
CACs = [[0 for _ in cost_types] for _ in tiers]
for i in range(len(tiers)):
    for j,cost_type in enumerate(cost_types):
        CACs[i][j] = tiers_cost[i][cost_type]/number_cus_last[0][i][cost_type]
CACs = pd.DataFrame(CACs,columns=cost_types)
CACs["tiers"] = [1,2,3]
CACs.set_index("tiers")

,email,social,display,paid_search,referral,organic_search,direct
tiers,,,,,,,
1,3.558719,1.600000,1.893939,5.681818,0.825764,0.0,0.0
2,3.603604,1.886792,2.444988,5.763689,0.949217,0.0,0.0
3,4.580153,2.352941,3.171247,6.237006,1.112760,0.0,0.0


In [9]:
# Compute the CAC for each tier each channel (position)
option = options["Position"]
for i,convert in enumerate(convert_TF):
    for j,tier in enumerate(tiers):
        allocation_group[i][j].apply(lambda x: count(x,number_cus_position[i][j],option),axis=1)
# Compute the CAC for each tier each channel
CACs = [[0 for _ in cost_types] for _ in tiers]
for i in range(len(tiers)):
    for j,cost_type in enumerate(cost_types):
        CACs[i][j] = tiers_cost[i][cost_type]/number_cus_position[0][i][cost_type]
CACs = pd.DataFrame(CACs,columns=cost_types)
CACs["tiers"] = [1,2,3]
CACs.set_index("tiers")

,email,social,display,paid_search,referral,organic_search,direct
tiers,,,,,,,
1,3.813883,1.923693,2.169825,7.095553,0.695894,0.0,0.0
2,4.332443,2.255724,2.675585,7.720021,0.790816,0.0,0.0
3,4.927457,2.810743,3.413876,8.045052,0.959969,0.0,0.0


We chose position, last, and linear based model to calculate CAC. No matter which method is used to calculate the CAC, the CAC of the paid search and email channels is significantly larger than the CAC of other channels. The CAC of referral is the smallest among all methods. As a company that sells directly to consumers, referal marketing itself is the most effective sales method, because first, consumers tend to trust people they know more. Second, it is easier to locate the correct consumer group, because people who know each other often have similarities. Third, the speed of information dissemination is fast. In the current digital age, whether it is through social media with familiar people or influential opinion leaders on the Internet, the dissemination of information is rapid and massive.
So we initially believe that we will allocate more budget to referral channel and less budget to paid-search and email channels,if each channel brings the same consumer value by default.


## Part 2, Allocation

In [10]:
import itertools

In [11]:
CACs_attr = CACs.drop(columns = ['direct', 'organic_search','tiers'])
CACs_attr

,email,social,display,paid_search,referral
0,3.813883,1.923693,2.169825,7.095553,0.695894
1,4.332443,2.255724,2.675585,7.720021,0.790816
2,4.927457,2.810743,3.413876,8.045052,0.959969


First, we calculate the marginal CACs.

In [29]:
CACs_marginal = CACs_attr
tier2_margin = CACs_marginal.iloc[1,]*2-CACs_marginal.iloc[0,]
tier3_margin = CACs_marginal.iloc[2,]*3-CACs_marginal.iloc[1,]*2

In [30]:
CACs_marginal.iloc[1,] = tier2_margin
CACs_marginal.iloc[2,] = tier3_margin
CACs_marginal

,email,social,display,paid_search,referral
0,3.813883,1.923693,2.169825,7.095553,0.695894
1,4.851003,2.587754,3.181346,8.344488,0.885738
2,6.117485,3.920782,4.890456,8.695116,1.298276


Next, we look for the optimal allocation method. 

In [13]:
# Get combinations of investment amount that number equals 5 and sum equals 11

alloc_tiers = [0,1,2,3]
combinations = list(itertools.combinations_with_replacement(alloc_tiers,5))
choices = [combination for combination in combinations if sum(combination) == 11]

In [14]:
# Get all possible allocation chocies by permuting possible investment amounts

alloc_choices = []
for choice in choices:
    permutations = list(set(itertools.permutations(choice)))
    alloc_choices.extend(permutations)

In [15]:
# Write a function that calculates the average CAC for allocation choices

def cac_calculator(alloc):
    
    '''
    The input of the function is a list of allocation.
    The output of the function is the calculated average CAC.
    '''
    alloc_sum = 0
    for i in range(len(alloc)):
        alloc_channel = alloc[i]
        index = alloc_channel - 1
        cac = CACs_attr.iloc[index,i]
        alloc_weight = cac * alloc_channel
        alloc_sum += alloc_weight

    avg_cac = alloc_sum/11
    
    return avg_cac

In [16]:
# Calculate the CAC for all choices and find the optimal one

all_cac = [cac_calculator(alloc) for alloc in alloc_choices]
cac_min = min(all_cac)
index_min = all_cac.index(cac_min)
optimal_allocation = list(alloc_choices[index_min])
print("Our optimal allocation method is {}.".format([alloc*1000 for alloc in optimal_allocation]))
print("CAC of this allocation method is {:.2f}.".format(cac_min))

Our optimal allocation method is [2000, 3000, 3000, 0, 3000].
CAC of this allocation method is 2.75.


In [17]:
# Focusing on tier 3

CACs_attr_2 = CACs[CACs['tiers'] == 3]
CACs_attr_2 = CACs_attr_2.drop(columns = ['direct', 'organic_search','tiers'])
CACs_attr_2

,email,social,display,paid_search,referral
2,4.927457,2.810743,3.413876,8.045052,0.959969


In [18]:
(CACs_attr_2.iloc[0,0]*2 + CACs_attr_2.iloc[0,1]*3 + CACs_attr_2.iloc[0,2]*3 + CACs_attr_2.iloc[0,4]*3)/11

2.8553343628011816

In [19]:
CACs_attr_2.iloc[0,4]

0.9599692809829875

In [20]:
# Write a function that calculates the average CAC for allocation choices

def cac_calculator2(alloc, attr):
    
    '''
    The input of the function is a list of allocation.
    The output of the function is the calculated average CAC.
    '''
    alloc_sum = 0
    for i in range(len(alloc)):
        alloc_channel = alloc[i]
        index = 0
        cac = attr.iloc[index,i]
        alloc_weight = cac * alloc_channel
        alloc_sum += alloc_weight

    avg_cac = alloc_sum/11
    
    return avg_cac

In [21]:
# Calculate the CAC for all choices and find the optimal one

all_cac = [cac_calculator2(alloc, CACs_attr_2) for alloc in alloc_choices]
cac_min = min(all_cac)
index_min = all_cac.index(cac_min)
optimal_allocation = list(alloc_choices[index_min])
print("Our optimal allocation method is {}.".format([alloc*1000 for alloc in optimal_allocation]))
print("CAC of this allocation method is {:.2f}.".format(cac_min))

Our optimal allocation method is [2000, 3000, 3000, 0, 3000].
CAC of this allocation method is 2.86.
